In [1]:
import pandas as pd

from plotly.offline import init_notebook_mode
from sqlalchemy import create_engine

from query_flow.parsers.postgres_parser import PostgresParser
from query_flow.vizualizers.query_vizualizer import QueryVizualizer

In [2]:
%load_ext autoreload
%autoreload 2

init_notebook_mode(connected=True)

In [3]:
con_str = 'postgresql:///etrabelsi_thesis'
query_renderer = QueryVizualizer(parser=PostgresParser())

## Identifying duplications

In [4]:
query = """
SELECT *
FROM (SELECT *,
             row_number() OVER (PARTITION BY titles.title_id) as row_num
      FROM titles
      INNER JOIN generes
              ON generes.genere_name LIKE '%'|| titles.genres || '%'
      WHERE titles.premiered >= date_part('year', CURRENT_DATE) - 2 AND
            generes.kid_safe = True
     ) as t
WHERE row_num = 1
"""

In [5]:
query = """
SELECT *
FROM titles
INNER JOIN generes
        ON position(generes.genere_name in titles.genres) > 0
WHERE titles.premiered >= date_part('year', CURRENT_DATE) - 2 AND
      generes.kid_safe = True
"""

In [6]:
flow_df = query_renderer.get_flow_df(query, con_str)
query_renderer.vizualize(flow_df, title="Identifying duplications", metrics=["actual_rows"], open_=False)

In [7]:
query = """
SELECT *
FROM (SELECT *,
             row_number() OVER (PARTITION BY titles.title_id) as row_num
      FROM titles
      INNER JOIN generes
              ON generes.genere_name LIKE '%'|| titles.genres || '%'
      WHERE titles.premiered >= date_part('year', CURRENT_DATE) - 2  AND
            generes.kid_safe = True
     ) as t
WHERE row_num = 1
"""

In [8]:
query = """
SELECT t.*
FROM (SELECT *,
             row_number() OVER (PARTITION BY titles.title_id) as row_num
      FROM titles
      INNER JOIN generes
              ON position(generes.genere_name in titles.genres) > 0
      WHERE titles.premiered >= date_part('year', CURRENT_DATE) - 2  AND
            generes.kid_safe = True
     ) as t
WHERE t.row_num = 1
"""

In [9]:
flow_df = query_renderer.get_flow_df(query, con_str)
query_renderer.vizualize(flow_df, title="Identifying duplications", metrics=["actual_rows"], open_=False)